<a href="https://colab.research.google.com/github/LEAF-BoiseState/Idaho-Mesonet-Design/blob/main/Idaho_TopoVeg_Clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
try:
    import wxee
except ImportError:
    print('wxee package not installed. Installing...')
    subprocess.check_call(['python','-m','pip','install','wxee'])
  #!pip install eemont

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from google.colab import drive
drive.mount('/content/gdrive')

import wxee

wxee package not installed. Installing...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xZSz5tO-7GKOWTyP0-ynhb_zyOEMrLgJXM0XSwhpzHM&tc=UOtyRvDvqCWnmie_rQV8wIgjDbCY2jD4o2kQoWR2nz8&cc=Rz7qJJa3Z-QR3whejmIaAYpdiLBlMSyXg1mLgrNXlQw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlb2g8Su-TqWBET6YjoEUc4WeBSGKlYdd9lCESyuJU6mRYdh-8EjeE

Successfully saved authorization token.
Mounted at /content/gdrive


In [ ]:
topoveg_write_path = '_Idaho_TopoVeg'

idaho_topo_tif = 'ID_elevation'
idaho_tcc_tif = 'ID_tcc'


In [ ]:
# Get the USGS HUC08 collection from the data catalog
us_states_tiger = ee.FeatureCollection('TIGER/2018/States')

id_outline = us_states_tiger.filter(ee.Filter.inList('STATEFP', ee.List(['16'])))

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(47.5, -116.25),zoom=5)

Map.addLayer(ee.Image().paint(id_outline, 0, 2), {}, 'Idaho')

Map.addLayerControl()
Map

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
idaho_elev = ee.Image("WWF/HydroSHEDS/03CONDEM").clip(id_outline.geometry())

#.wx.to_xarray(path=prism_write_path+idaho_norm_nc, region=id_outline.geometry(), scale=92.77)

elevationVis = {
  'min': -50.0,
  'max': 3000.0,
  'gamma': 2.0,
}
Map2 = emap.Map(center=(47.5, -116.25),zoom=5)
Map2.addLayer(idaho_elev, elevationVis, 'Elevation')

Map2.addLayerControl()
Map2

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
downConfig = {'scale': 92.77, "maxPixels": 1.0E13, 'driveFolder': topoveg_write_path }  # scale means resolution.

task = ee.batch.Export.image(idaho_elev.clip(id_outline.geometry()), idaho_topo_tif, downConfig)
task.start()
print(task.status())


{'state': 'READY', 'description': 'ID_elevation', 'creation_timestamp_ms': 1697428465429, 'update_timestamp_ms': 1697428465429, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'H5IQWB6BQ4GMPOH7ALIF4NMK', 'name': 'projects/earthengine-legacy/operations/H5IQWB6BQ4GMPOH7ALIF4NMK'}


In [ ]:
print(task.status())

{'state': 'READY', 'description': 'ID_elevation', 'creation_timestamp_ms': 1697428465429, 'update_timestamp_ms': 1697428465429, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'H5IQWB6BQ4GMPOH7ALIF4NMK', 'name': 'projects/earthengine-legacy/operations/H5IQWB6BQ4GMPOH7ALIF4NMK'}


In [ ]:
idaho_tcc = ee.ImageCollection("NASA/MEASURES/GFCC/TC/v3").filterBounds(id_outline.geometry()) \
  .filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('tree_canopy_cover').map(lambda image: image.clip(id_outline.geometry()))

tccVis = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'afce56', '5f9c00', '0e6a00', '003800'],
}
Map3 = emap.Map(center=(47.5, -116.25),zoom=5)
Map3.addLayer(idaho_tcc, tccVis, 'Tree Canopy Cover')

Map3.addLayerControl()
Map3

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
idaho_tcc_mosaic = idaho_tcc.mosaic()
idaho_tcc_mosaic = idaho_tcc_mosaic.unmask(-9999)
idaho_tcc_mosaic = idaho_tcc_mosaic.clip(id_outline.geometry().bounds())

tccVis = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'afce56', '5f9c00', '0e6a00', '003800'],
}
Map4 = emap.Map(center=(47.5, -116.25),zoom=5)
Map4.addLayer(idaho_tcc_mosaic, tccVis, 'Tree Canopy Cover')

Map4.addLayerControl()
Map4

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': '_Idaho_TopoVeg' }  # scale means resolution.

task = ee.batch.Export.image(idaho_tcc_mosaic, idaho_tcc_tif, downConfig)
task.start()
print(task.status())


{'state': 'READY', 'description': 'ID_tcc', 'creation_timestamp_ms': 1697428491590, 'update_timestamp_ms': 1697428491590, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GQSSGZ6NAK45AAQA2KGJC3XB', 'name': 'projects/earthengine-legacy/operations/GQSSGZ6NAK45AAQA2KGJC3XB'}


In [ ]:
task

<Task GQSSGZ6NAK45AAQA2KGJC3XB Type.EXPORT_IMAGE: ID_tcc (State.UNSUBMITTED)>